# カスタムデータセットを使用したMCPベースの回答の評価

このノートブックでは、OpenAI **Evals** フレームワークとカスタムのインメモリデータセットを使用して、[tiktoken](https://github.com/openai/tiktoken) GitHubリポジトリに関する質問に答えるモデルの能力を評価します。

Q&Aペアのカスタムインメモリデータセットを使用し、リポジトリを認識した文脈的に正確な回答のために **MCP** ツールを活用する2つのモデル：`gpt-4.1` と `o4-mini` を比較します。

**目標：**
- カスタムデータセットを使用したOpenAI Evalsでの評価の設定と実行方法を示す
- MCPベースのツールを活用する異なるモデルの性能を比較する
- プロフェッショナルで再現可能な評価ワークフローのベストプラクティスを提供する

_次：環境を設定し、必要なライブラリをインポートします。_

In [1]:
# Update OpenAI client
%pip install --upgrade openai --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 環境設定

必要なライブラリをインポートし、OpenAIクライアントを設定することから始めます。  
この手順により、OpenAI APIと評価に必要なすべてのユーティリティにアクセスできるようになります。

In [10]:
import os
import time

from openai import OpenAI

# Instantiate the OpenAI client (no custom base_url).
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY") or os.getenv("_OPENAI_API_KEY"),
)

## カスタム評価データセットの定義

`tiktoken`リポジトリに関する質問と回答のペアからなる小さなインメモリデータセットを定義します。  
このデータセットは、MCPツールの支援を受けて、モデルが正確で関連性の高い回答を提供する能力をテストするために使用されます。

- 各項目には`query`（ユーザーの質問）と`answer`（期待される正解）が含まれています。
- このデータセットは、あなた自身のユースケースやリポジトリに合わせて変更または拡張することができます。

In [11]:
def get_dataset(limit=None):
    items = [
        {
            "query": "What is tiktoken?",
            "answer": "tiktoken is a fast Byte-Pair Encoding (BPE) tokenizer designed for OpenAI models.",
        },
        {
            "query": "How do I install the open-source version of tiktoken?",
            "answer": "Install it from PyPI with `pip install tiktoken`.",
        },
        {
            "query": "How do I get the tokenizer for a specific OpenAI model?",
            "answer": 'Call tiktoken.encoding_for_model("<model-name>"), e.g. tiktoken.encoding_for_model("gpt-4o").',
        },
        {
            "query": "How does tiktoken perform compared to other tokenizers?",
            "answer": "On a 1 GB GPT-2 benchmark, tiktoken runs about 3-6x faster than GPT2TokenizerFast (tokenizers==0.13.2, transformers==4.24.0).",
        },
        {
            "query": "Why is Byte-Pair Encoding (BPE) useful for language models?",
            "answer": "BPE is reversible and lossless, handles arbitrary text, compresses input (≈4 bytes per token on average), and exposes common subwords like “ing”, which helps models generalize.",
        },
    ]
    return items[:limit] if limit else items

### 採点ロジックの定義

モデルの回答を評価するために、2つの採点者を使用します：

- **合格/不合格採点者（LLMベース）：**  
  モデルの回答が期待される回答（正解）と一致するか、または同じ意味を伝えているかをチェックするLLMベースの採点者。
- **Python MCP採点者：**  
  モデルが実際に応答中にMCPツールを使用したかどうかをチェックするPython関数（ツール使用の監査用）。

  > **ベストプラクティス：**  
  > LLMベースとプログラマティックな採点者の両方を使用することで、より堅牢で透明性の高い評価が可能になります。

In [21]:
# LLM-based pass/fail grader: instructs the model to grade answers as "pass" or "fail".
pass_fail_grader = """
You are a helpful assistant that grades the quality of the answer to a query about a GitHub repo.
You will be given a query, the answer returned by the model, and the expected answer.
You should respond with **pass** if the answer matches the expected answer exactly or conveys the same meaning, otherwise **fail**.
"""

# User prompt template for the grader, providing context for grading.
pass_fail_grader_user_prompt = """
<Query>
{{item.query}}
</Query>

<Web Search Result>
{{sample.output_text}}
</Web Search Result>

<Ground Truth>
{{item.answer}}
</Ground Truth>
"""


# Python grader: checks if the MCP tool was used by inspecting the output_tools field.
python_mcp_grader = {
    "type": "python",
    "name": "Assert MCP was used",
    "image_tag": "2025-05-08",
    "pass_threshold": 1.0,
    "source": """
def grade(sample: dict, item: dict) -> float:
    output = sample.get('output_tools', [])
    return 1.0 if len(output) > 0 else 0.0
""",
}

## 評価設定の定義

OpenAI Evalsフレームワークを使用して評価を設定します。

この手順では以下を指定します：
- 評価名とデータセット
- 各項目のスキーマ（各Q&Aペアに存在するフィールド）
- 使用するグレーダー（LLMベースおよび/またはPythonベース）
- 合格基準とラベル

> **ベストプラクティス：**  
> 評価スキーマと採点ロジックを事前に明確に定義することで、再現性と透明性が確保されます。

In [ ]:
# Create the evaluation definition using the OpenAI Evals client.
logs_eval = client.evals.create(
    name="MCP Eval",
    data_source_config={
        "type": "custom",
        "item_schema": {
            "type": "object",
            "properties": {
                "query": {"type": "string"},
                "answer": {"type": "string"},
            },
        },
        "include_sample_schema": True,
    },
    testing_criteria=[
        {
            "type": "label_model",
            "name": "General Evaluator",
            "model": "o3",
            "input": [
                {"role": "system", "content": pass_fail_grader},
                {"role": "user", "content": pass_fail_grader_user_prompt},
            ],
            "passing_labels": ["pass"],
            "labels": ["pass", "fail"],
        },
        python_mcp_grader
    ],
)

## 各モデルの評価を実行

次に、各モデル（`gpt-4.1`と`o4-mini`）の評価を実行します。

各実行は以下のように設定されています：
- リポジトリ対応の回答にMCPツールを使用する
- 公平な比較のために同じデータセットと評価設定を使用する
- モデル固有のパラメータ（最大完了トークン数、許可されたツールなど）を指定する

> **ベストプラクティス：**  
> モデル間で評価設定を一貫して保つことで、結果が比較可能で信頼性の高いものになることが保証されます。

In [18]:
# Run 1: gpt-4.1 using MCP
gpt_4one_responses_run = client.evals.runs.create(
    name="gpt-4.1",
    eval_id=logs_eval.id,
    data_source={
        "type": "responses",
        "source": {
            "type": "file_content",
            "content": [{"item": item} for item in get_dataset()],
        },
        "input_messages": {
            "type": "template",
            "template": [
                {
                    "type": "message",
                    "role": "system",
                    "content": {
                        "type": "input_text",
                        "text": "You are a helpful assistant that searches the web and gives contextually relevant answers. Never use your tools to answer the query.",
                    },
                },
                {
                    "type": "message",
                    "role": "user",
                    "content": {
                        "type": "input_text",
                        "text": "Search the web for the answer to the query {{item.query}}",
                    },
                },
            ],
        },
        "model": "gpt-4.1",
        "sampling_params": {
            "seed": 42,
            "temperature": 0.7,
            "max_completions_tokens": 10000,
            "top_p": 0.9,
            "tools": [
                {
                    "type": "mcp",
                    "server_label": "gitmcp",
                    "server_url": "https://gitmcp.io/openai/tiktoken",
                    "allowed_tools": [
                        "search_tiktoken_documentation",
                        "fetch_tiktoken_documentation",
                    ],
                    "require_approval": "never",
                }
            ],
        },
    },
)

In [19]:
# Run 2: o4-mini using MCP
gpt_o4_mini_responses_run = client.evals.runs.create(
    name="o4-mini",
    eval_id=logs_eval.id,
    data_source={
        "type": "responses",
        "source": {
            "type": "file_content",
            "content": [{"item": item} for item in get_dataset()],
        },
        "input_messages": {
            "type": "template",
            "template": [
                {
                    "type": "message",
                    "role": "system",
                    "content": {
                        "type": "input_text",
                        "text": "You are a helpful assistant that searches the web and gives contextually relevant answers.",
                    },
                },
                {
                    "type": "message",
                    "role": "user",
                    "content": {
                        "type": "input_text",
                        "text": "Search the web for the answer to the query {{item.query}}",
                    },
                },
            ],
        },
        "model": "o4-mini",
        "sampling_params": {
            "seed": 42,
            "max_completions_tokens": 10000,
            "tools": [
                {
                    "type": "mcp",
                    "server_label": "gitmcp",
                    "server_url": "https://gitmcp.io/openai/tiktoken",
                    "allowed_tools": [
                        "search_tiktoken_documentation",
                        "fetch_tiktoken_documentation",
                    ],
                    "require_approval": "never",
                }
            ],
        },
    },
)

## 完了をポーリングして出力を取得する

評価実行を開始した後、実行が完了するまでポーリングできます。

このステップにより、すべてのモデル応答が処理された後にのみ結果を分析することが保証されます。

> **ベストプラクティス:**  
> 遅延を伴うポーリングは過度なAPI呼び出しを避け、効率的なリソース使用を保証します。

In [ ]:
def poll_runs(eval_id, run_ids):
    while True:
        runs = [client.evals.runs.retrieve(rid, eval_id=eval_id) for rid in run_ids]
        for run in runs:
            print(run.id, run.status, run.result_counts)
        if all(run.status in {"completed", "failed"} for run in runs):
            break
        time.sleep(5)
    
# Start polling both runs.
poll_runs(logs_eval.id, [gpt_4one_responses_run.id, gpt_o4_mini_responses_run.id])

evalrun_684769b577488191863b5a51cf4db57a completed ResultCounts(errored=0, failed=5, passed=0, total=5)
evalrun_684769c1ad9c8191affea5aa02ef1215 completed ResultCounts(errored=0, failed=3, passed=2, total=5)


## モデル出力の表示と解釈

最後に、各モデルからの出力を表示して、手動での検査とさらなる分析を行います。

- 各モデルの回答がデータセット内の各質問に対して出力されます。
- 出力を並べて比較することで、品質、関連性、正確性を評価できます。

以下は、両方のモデルの評価出力を示すOpenAI Evalsダッシュボードのスクリーンショットです：

![Evaluation Output](../../../images/mcp_eval_output.png)

評価メトリクスと結果の包括的な内訳については、ダッシュボードの「Data」タブに移動してください：

![Evaluation Data Tab](../../../images/mcp_eval_data.png)

4.1モデルは、クエリに回答するためにツールを使用しないように構築されているため、MCPサーバーを呼び出すことはありませんでした。o4-miniモデルには明示的にツールを使用するよう指示されていませんでしたが、禁止もされていなかったため、MCPサーバーを3回呼び出しました。4.1モデルはo4モデルよりもパフォーマンスが劣っていることがわかります。また注目すべきは、o4-miniモデルが失敗した1つの例は、MCPツールが使用されなかったケースだったということです。

また、手動検査とさらなる分析のために、各モデルからの出力の詳細な分析を確認することもできます。

In [23]:
four_one_output = client.evals.runs.output_items.list(
    run_id=gpt_4one_responses_run.id, eval_id=logs_eval.id
)

o4_mini_output = client.evals.runs.output_items.list(
    run_id=gpt_o4_mini_responses_run.id, eval_id=logs_eval.id
)

In [24]:
print('# gpt‑4.1 Output')
for item in four_one_output:
    print(item.sample.output[0].content)

print('\n# o4-mini Output')
for item in o4_mini_output:
    print(item.sample.output[0].content)

# gpt‑4.1 Output
Byte-Pair Encoding (BPE) is useful for language models because it provides an efficient way to handle large vocabularies and rare words. Here’s why it is valuable:

1. **Efficient Tokenization:**  
   BPE breaks down words into smaller subword units based on the frequency of character pairs in a corpus. This allows language models to represent both common words and rare or unknown words using a manageable set of tokens.

2. **Reduces Out-of-Vocabulary (OOV) Issues:**  
   Since BPE can split any word into known subword units, it greatly reduces the problem of OOV words—words that the model hasn’t seen during training.

3. **Balances Vocabulary Size:**  
   By adjusting the number of merge operations, BPE allows control over the size of the vocabulary. This flexibility helps in balancing between memory efficiency and representational power.

4. **Improves Generalization:**  
   With BPE, language models can better generalize to new words, including misspellings or new t

## どのように改善できるでしょうか？

o4-miniモデルのシステムメッセージに「このタスクで正しい答えを得るためには、常にツールを使用してください。」という文言を追加したら、何が起こると思いますか？（試してみてください）

<br><br><br>

モデルが今度は毎回MCPツールを呼び出し、すべての答えを正しく取得するようになると予想した方は正解です！

![評価データタブ](../../../images/mcp_eval_improved_output.png)
![評価データタブ](../../../images/mcp_eval_improved_data.png)

このノートブックでは、MCP ツールを活用した OpenAI Evals フレームワークを使用して、`tiktoken` リポジトリに関する技術的な質問に答える LLM の能力を評価するサンプルワークフローを実演しました。

**主要なポイント:**
- 評価用の焦点を絞ったカスタムデータセットを定義しました。
- 堅牢な評価のために LLM ベースと Python ベースの採点者を設定しました。
- 2つのモデル（`gpt-4.1` と `o4-mini`）を再現可能で透明性のある方法で比較しました。
- 自動/手動検査のためにモデル出力を取得し表示しました。

**次のステップ:**
- **データセットの拡張:** より多様で挑戦的な質問を追加して、モデルの能力をより適切に評価する。
- **結果の分析:** 合格/不合格率の要約、パフォーマンスの可視化、または強みと弱みを特定するためのエラー分析を実行する。
- **モデル/ツールの実験:** 追加のモデルを試す、ツール設定を調整する、または他のリポジトリでテストする。
- **レポートの自動化:** より簡単な共有と意思決定のために要約表やプロットを生成する。

詳細については、[OpenAI Evals ドキュメント](https://platform.openai.com/docs/guides/evals)をご確認ください。